In [ ]:
%pwd

: 

In [3]:
import os

In [4]:
os.chdir("../")

In [5]:
%pwd

'/workspaces/MLOps-classification'

In [6]:
from dataclasses import dataclass, field
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    """Configuration for data ingestion."""
    data_dir: Path
    """Path to the directory containing the data."""
    source_URL: str
    """URL of the data source."""
    local_data_file: Path
    """Path to the local file containing the data."""
    unzip_dir: Path
    """Path to the directory containing the unzipped data."""

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

SyntaxError: invalid syntax (3371224337.py, line 1)